<a href="https://colab.research.google.com/github/wundercomm/Speakwize/blob/main/tools/podcast-precess/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detect GPU

In [1]:
!nvidia-smi

Mon Sep  4 19:38:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install CondaColab

In [2]:
%pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


# Check CondaColab it's ok

In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
podcast_audio_directory = '/content/drive/MyDrive/AI/podcast/audio'
podcast_text_directory = '/content/drive/MyDrive/AI/podcast/text'
podcast_stats_csv = '/content/drive/MyDrive/AI/podcast/stats.csv'
whisper_model = 'medium'
decode_options = {"language": "es"}

In [ ]:
!mkdir -p {podcast_audio_directory}
!mkdir -p {podcast_text_directory}

# Clone Repo

In [7]:
!git clone https://github.com/wundercomm/Speakwize

Cloning into 'Speakwize'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 314 (delta 164), reused 231 (delta 90), pack-reused 0
Receiving objects: 100% (314/314), 1.82 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (164/164), done.


# Install my env

In [ ]:
!mamba env update -n base -f /content/podcast-coffeebreak/environment.yml

# Check youtube-dl Version

In [ ]:
!youtube-dl --version

# Download Podcast

In [ ]:
!bash /content/podcast-coffeebreak/utils/000_download_podcast.sh -u 'https://podcasts.google.com/feed/aHR0cDovL2ZlZWRzLmZlZWRidXJuZXIuY29tL1BvZGNhc3RDb2ZmZWVCcmVhaw?sa=X&ved=0CD4Q9sEGahcKEwjw1Zv4i8_6AhUAAAAAHQAAAAAQEw' -d /content/drive/MyDrive/AI/podcast/audio

In [ ]:
import os
import whisper
import json
import time
import pandas as pd


df = pd.DataFrame(columns=
        [
            'id',
            'filename',
            'audio_filename',
            'text_filename',
            'size_in_bytes',
            'size_in_sconds',
            'transcribe_status',
            'transcribe_start',
            'transcribe_end',
            'transcribe_duration',
            'transcribe_model',
            'transcribe_language'
        ]
)
df = df.set_index('id')
model = whisper.load_model(whisper_model)

for filename in sorted(os.listdir(podcast_audio_directory)):
    id=filename.split('.')[0] + "_" + whisper_model
    audio_filename = os.path.join(podcast_audio_directory, filename)
    if not whisper_model == "medium":
        text_filename = os.path.join(podcast_text_directory, filename.split('.')[0] + "_" + whisper_model + '.json')
    else:
        text_filename = os.path.join(podcast_text_directory, filename.split('.')[0] + '.json')
    size_in_bytes = os.path.getsize(audio_filename)
    # size_in_sconds=audio.info.length
    df.loc[id]={
            "id": id,
            "filename": filename.split('.')[0],
            "audio_filename": audio_filename,
            "text_filename": text_filename,
            "size_in_bytes": size_in_bytes,
            # "size_in_sconds": size_in_sconds
    }
    if not os.path.exists(text_filename):
        if os.path.isfile(audio_filename):
            transcribe_start = time.time()
            print('\r transcribing {0} at {1}'.format(audio_filename, transcribe_start))
            result = model.transcribe(audio_filename, **decode_options)
            transcribe_end = time.time()
            transcribe_duration = transcribe_end - transcribe_start
            transcribe_status = "1"
            transcribe_model=model
            with open(text_filename, 'w',encoding='utf8') as text_filename_file:
                print('\r Writing {0}'.format(text_filename))
                json.dump(result, text_filename_file, indent=4)
            print('\r end transcribe {0} at {1} duration of {2} sec'.format(audio_filename, transcribe_end, transcribe_duration))
            df.loc[id]={
                "transcribe_status": transcribe_status,
                "transcribe_start": transcribe_start,
                "transcribe_end": transcribe_end,
                "transcribe_duration": transcribe_duration,
                "transcribe_model": whisper_model,
                "transcribe_language": decode_options["language"]
            }
    df.to_csv(podcast_stats_csv, mode='a', index=True, header=True)
    print("\r {0} Data appended successfully".format(id))

SyntaxError: invalid syntax (2718330330.py, line 46)